# Read in data

In [101]:
import pandas as pd
import numpy as np


def try_convert_str(x):
    try:
        intx = int(x)
        return str(intx)
    except:
        return np.nan
    

def try_convert_float(x):
    try:
        x = x.replace(",", "")
        floatx = float(x)
        return floatx
    except:
        return np.nan


# Read in county pres vote data
county_pres = pd.read_csv("../data/raw/countypres_2000-2020.csv")
county_pres['county_fips'] = county_pres['county_fips'].apply(try_convert_str)
county_pres = county_pres.dropna(subset=['county_fips'])
county_pres = county_pres.query("year == 2020")
df_wide_filled = county_pres.pivot_table(index=["year", "state", "county_name", "county_fips", "totalvotes"],
                         columns="party", values="candidatevotes").reset_index()
df_wide_filled = df_wide_filled.fillna(0)
df_wide_filled['sum'] = (
    df_wide_filled['DEMOCRAT'] +
    df_wide_filled['REPUBLICAN'] +
    df_wide_filled['LIBERTARIAN'] + 
    df_wide_filled['GREEN'] +
    df_wide_filled['OTHER']
)
for col in ['DEMOCRAT', 'REPUBLICAN', 'LIBERTARIAN', 'GREEN', 'OTHER']:
    df_wide_filled[col] = df_wide_filled[col].astype(int)
df_wide_filled['error'] = df_wide_filled['sum'] - df_wide_filled['totalvotes']
###############

# Read in census data
###############
census = pd.read_csv("../data/raw/PopulationEstimates.csv")
census['FIPStxt'] = census['FIPStxt'].astype(str)
for col in census.columns:
    if col not in ['FIPStxt', 'State', 'Area_Name', 'Rural_Urban_Continuum_Code_2013',
       'Rural_Urban_Continuum_Code_2023',
       'Economic_typology_2015']:
        census[col] = census[col].apply(try_convert_float)
census.columns = [x.lower() for x in census.columns]
census = census.query("area_name != 'United States'")
###############


print(sorted(census.columns))
print()
print(sorted(df_wide_filled.columns))


['area_name', 'births_2020', 'births_2021', 'births_2022', 'births_2023', 'census_2020_pop', 'deaths_2020', 'deaths_2021', 'deaths_2022', 'deaths_2023', 'domestic_mig_2020', 'domestic_mig_2021', 'domestic_mig_2022', 'domestic_mig_2023', 'economic_typology_2015', 'estimates_base_2020', 'fipstxt', 'gq_estimates_2020', 'gq_estimates_2021', 'gq_estimates_2022', 'gq_estimates_2023', 'gq_estimates_base_2020', 'international_mig_2020', 'international_mig_2021', 'international_mig_2022', 'international_mig_2023', 'n_pop_chg_2020', 'n_pop_chg_2021', 'n_pop_chg_2022', 'n_pop_chg_2023', 'natural_chg_2020', 'natural_chg_2021', 'natural_chg_2022', 'natural_chg_2023', 'net_mig_2020', 'net_mig_2021', 'net_mig_2022', 'net_mig_2023', 'pop_estimate_2020', 'pop_estimate_2021', 'pop_estimate_2022', 'pop_estimate_2023', 'r_birth_2021', 'r_birth_2022', 'r_birth_2023', 'r_death_2021', 'r_death_2022', 'r_death_2023', 'r_domestic_mig_2021', 'r_domestic_mig_2022', 'r_domestic_mig_2023', 'r_international_mig_202

# Merge data

In [108]:
merged = pd.merge(df_wide_filled, census, left_on="county_fips", right_on="fipstxt", how="left", indicator="merge_indicator")
merged['crude_turnout'] = merged['totalvotes'] / merged['pop_estimate_2023']
print(merged['merge_indicator'].value_counts())
merged_both = merged.query("merge_indicator == 'both'")
merged_both.to_csv("../data/clean/county_pres_census_2020.csv", index=False)

merge_indicator
both          3115
left_only       39
right_only       0
Name: count, dtype: int64


# Filter for large counties

In [112]:
top_20_counties = merged.nlargest(20, 'pop_estimate_2023')
top_20_counties[['county_name', 'state_x', 'pop_estimate_2023', 'totalvotes', 'DEMOCRAT', 'REPUBLICAN', 'crude_turnout']]
top_20_counties = top_20_counties.rename(columns={"state_x": "state"})
top_20_counties.to_csv("../data/clean/top_20_counties.csv", index=False)

In [136]:

# Rank county subsets by: total population, absolute difference in votes between Republicans and Democrats, and total voter turnout
########################
def rank_county_subsets(data, n):
    from itertools import combinations
    from scipy.stats import entropy
    import numpy as np
    
    top_20 = data[['county_name', 'pop_estimate_2023', 'totalvotes', 'DEMOCRAT', 'REPUBLICAN', 'crude_turnout', 'state']]

    county_subsets = list(combinations(top_20.index, n))

    subset_results = []

    for subset in county_subsets:
        subset_data = top_20.loc[list(subset)]

        # Calculate total population
        total_population = subset_data['pop_estimate_2023'].sum()

        # Calculate total Democrat and Republican votes
        total_democrat_votes = subset_data['DEMOCRAT'].sum()
        total_republican_votes = subset_data['REPUBLICAN'].sum()
        
        # Calculate proportions of Democrat and Republican votes
        total_votes = total_democrat_votes + total_republican_votes
        dem_proportion = total_democrat_votes / total_votes
        rep_proportion = total_republican_votes / total_votes
        vote_distribution = np.array([dem_proportion, rep_proportion])
        vote_entropy = entropy(vote_distribution, base=2)  

        # Calculate total voter turnout as the average crude turnout across the subset
        avg_turnout = subset_data['crude_turnout'].mean()

        # Calculate number of states
        num_states = len(subset_data['state'].unique())

        # Store the results
        subset_results.append({
            'counties': tuple(subset_data['county_name']),
            'total_population': total_population,
            'vote_entropy': vote_entropy,
            'num_states': num_states,
            'avg_turnout': avg_turnout
        })

    subset_df = pd.DataFrame(subset_results)

    return subset_df

n = 5
subset_rankings_df = rank_county_subsets(top_20_counties, n)
for col in ['total_population', 'vote_entropy', 'avg_turnout', 'num_states']:
    subset_rankings_df[f"{col}_rank"] = subset_rankings_df[col].rank(ascending=False)


########################




# Weighting the counties

########################
weights = {
    'total_population': 0.25,
    'vote_entropy': 0.25,
    'num_states': 0.25,
    'avg_turnout': 0.25
}
subset_rankings_df['score'] = 0
for col in weights.keys():
    subset_rankings_df['score'] += subset_rankings_df[f"{col}_rank"] * weights[col]
########################

subset_rankings_df = subset_rankings_df.sort_values('score')
display(subset_rankings_df.head(5))


,counties,total_population,vote_entropy,num_states,avg_turnout,total_population_rank,vote_entropy_rank,avg_turnout_rank,num_states_rank,score
7128,"(HARRIS, MARICOPA, ORANGE, BROWARD, MIDDLESEX)",16143234.0,0.974558,5,0.459809,5285.0,1799.0,1839.0,2096.5,2754.875
7053,"(HARRIS, MARICOPA, ORANGE, MIAMI-DADE, MIDDLESEX)",16867570.0,0.984530,5,0.448323,4640.0,632.0,3696.0,2096.5,2766.125
7131,"(HARRIS, MARICOPA, ORANGE, WAYNE, MIDDLESEX)",15931872.0,0.971260,5,0.461858,5484.0,2269.0,1595.0,2096.5,2861.125
9804,"(MARICOPA, ORANGE, MIAMI-DADE, CLARK, KING)",15016446.0,0.977177,5,0.463199,6506.0,1420.0,1427.0,2096.5,2862.375
7045,"(HARRIS, MARICOPA, ORANGE, MIAMI-DADE, KING)",17514998.0,0.976820,5,0.447845,4181.0,1467.0,3795.0,2096.5,2884.875
